<a href="https://colab.research.google.com/github/pau-torrente/capstone-project/blob/pau/tidy_file_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Markdown


station_dataframe = dd.read_csv('/content/drive/MyDrive/CapstoneProject/data_bicing_joined_HX.csv', assume_missing=True, delimiter=';')

weather_dataframe = dd.read_csv('/content/drive/MyDrive/CapstoneProject/weather.csv', assume_missing=True, delimiter=',')

station_dataframe = station_dataframe.loc[station_dataframe['status'] == 'IN_SERVICE']

bare_df = station_dataframe[['station_id', 'lat', 'lon', 'year', 'month', 'day', 'hour', '% Docks Availlable',  '% Docks Available H-4','% Docks Available H-3', '% Docks Available H-2', '% Docks Available H-1']]
bare_df = bare_df.rename(columns={'% Docks Availlable': 'percentage'})
for i in range(1, 5):
    bare_df = bare_df.rename(columns={f'% Docks Available H-{i}': f'ctx-{i}'})

# Print the head of the updated DataFrame

bare_df['index'] = bare_df.index
bare_df = bare_df[['index', 'station_id', 'lat', 'lon', 'year', 'month', 'day', 'hour', 'ctx-4', 'ctx-3', 'ctx-2',	'ctx-1', 'percentage']]

bare_df.head()

,index,station_id,lat,lon,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,percentage
0,0,290.0,41.437338,2.174096,2019.0,7.0,22.0,8.0,0.352941,0.352941,0.352941,0.504902,0.751131
1,1,271.0,41.450608,2.192363,2022.0,6.0,10.0,21.0,0.753968,0.659341,0.645022,0.686508,0.769231
2,2,149.0,41.395868,2.192952,2022.0,6.0,8.0,20.0,0.491582,0.675214,0.864198,0.801347,0.735043
3,3,342.0,41.403497,2.193658,2020.0,2.0,4.0,4.0,0.898990,0.831909,0.777778,0.777778,0.777778
4,4,358.0,41.387244,2.179304,2021.0,5.0,28.0,8.0,0.480000,0.480000,0.513333,0.766667,0.943333


In [11]:
data_2_predict = pd.read_csv('/content/drive/MyDrive/CapstoneProject/metadata_sample_submission.csv', delimiter=',')

data_2_predict.head()

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1
0,0,394,3,7,8,0.753086,0.780864,0.799383,0.824074
1,1,337,3,23,12,0.463768,0.536232,0.532609,0.601449
2,2,368,3,31,1,0.787037,0.709877,0.611111,0.601852
3,3,327,3,23,15,0.753472,0.809028,0.819444,0.736111
4,4,328,3,4,20,0.861111,0.802469,0.814815,0.827160


In [ ]:
len(bare_df)

14877478

In [5]:
def weather_prep(weather_df:dd) -> dd:
    
    weather = weather_df.copy()

    weather = weather.groupby(weather.index//2).mean()
    
    weather['mm_precip'] = weather['mm_precip']*2
    weather['timestamp'] = (weather['timestamp']-900).astype(int)
    weather['datetime'] = weather['timestamp'].map(lambda x: pd.to_datetime(x, unit='s'))

    
    weather['timestamp'] = weather['timestamp'].astype(int)

    weather['datetime'] = weather['timestamp'].map(lambda x: pd.to_datetime(x, unit='s'))
    
    return weather

def weather_merge(weather_df:dd, station_data:dd) -> dd:
    weather = weather_df.copy()
    stations = station_data.copy()

    stations[['year', 'month', 'day', 'hour']] = stations[['year', 'month', 'day', 'hour']].astype(int)

    stations['datetime'] = dd.to_datetime(stations['year'].astype(str) + '-' +
                                                stations['month'].astype(str) + '-' +
                                                stations['day'].astype(str) + ' ' +
                                                stations['hour'].astype(str) + ':00:00')
    
    stations = dd.merge(left=stations, right=weather[['datetime', 'temperature','mm_precip']], left_on='datetime', right_on='datetime', how='left')

    for i in range(1,5):
        df_weather_shifted = weather.copy()
        df_weather_shifted['datetime'] = df_weather_shifted['datetime'] + pd.Timedelta(hours=-i)
        stations = stations.merge(df_weather_shifted[['datetime', 'temperature','mm_precip']], on='datetime', how='inner', suffixes=('', f'-{abs(i)}'))

    return stations


def extra_time_info(df:dd) -> dd:

    def is_weekend(day_of_week):
        return 1 if day_of_week >= 5 else 0

    df['is_weekend'] = df['datetime'].dt.dayofweek.map(is_weekend, meta=('is_weekend', 'int64'))

    df['timeframe1'] = df['datetime'].dt.hour.map(lambda x: 1 if x <= 4 else 0, meta=('timeframe1', 'int64'))
    df['timeframe2'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 5 and x <=9 else 0, meta=('timeframe1', 'int64'))
    df['timeframe3'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 10 and x <=14 else 0, meta=('timeframe1', 'int64'))
    df['timeframe4'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 15 and x <=19 else 0, meta=('timeframe1', 'int64'))
    df['timeframe5'] = df['datetime'].dt.hour.map(lambda x: 1 if x >= 20 else 0, meta=('timeframe1', 'int64'))

    df.drop(['datetime'], axis = 1)
    
    return df


def station_loc(id_lat_lon:dd, df:dd) -> dd: #this should be applied for the 2023 march dataset to predict from station_id-location paris from february 2023

    assert all(item in list(id_lat_lon.columns) for item in ['station_id', 'lat', 'lon']), 'id_lat_lon must contain station_id, lat and lon columns'
    id_locator = id_lat_lon.copy()
    data = df.copy()
    id_locator = id_locator.drop_duplicates(subset=['station_id'])

    data = data.merge(id_locator[['station_id', 'lat', 'lon']], on='station_id', how='left')
    data = data.drop(['station_id'], axis=1)

    return data

def time_norm(df:dd, columns:list) -> dd:
    
        data = df.copy()
        for col in columns:
            data['cos_'+col] = np.cos(2*np.pi*data[col]/data[col].max())
            data['sin_'+col] = np.sin(2*np.pi*data[col]/data[col].max())
            data = data.drop([col], axis=1)

        if 'year' in data.columns:
            data['year_normed'] = (data['year']-2019)/(data['year'].max()-data['year'].min()+1)
            data.drop(['year'], axis = 1)

        elif 'year' not in data.columns:
            data['year_normed'] = 1
            
          
            
        return data




In [13]:
weather_prepped = weather_prep(weather_dataframe)

data_prepared = weather_merge(weather_prepped, bare_df)

data_prepared = extra_time_info(data_prepared)

data_prepared = data_prepared[data_prepared['hour'].isin([4, 9, 14, 19, 23])]

data_prepared = time_norm(df = data_prepared, columns = ['month', 'day', 'hour'])

data_prepared.drop(['station_id'], axis = 1)



# locations = station_dataframe[['station_id', 'lat', 'lon']]

# data_prepared = station_loc(id_lat_lon=locations, df=data_prepared)

Markdown(data_prepared.head().to_markdown())




KeyboardInterrupt: ignored

In [ ]:
from sklearn.preprocessing import MinMaxScaler, 

In [7]:
len(data_prepared)

14873854

In [ ]:
data_prepared = data_prepared[['year', 'month', 'day', 'hour', 'is_weekend', 'timeframe1',
       'timeframe2', 'timeframe3', 'timeframe4', 'timeframe5', 'lat', 'lon', 
       'temperature-4', 'mm_precip-4', 'ctx-4', 'temperature-3', 'mm_precip-3', 'ctx-3', 
       'temperature-2', 'mm_precip-2', 'ctx-2', 'temperature-1', 'mm_precip-1', 'ctx-1', 
       'temperature', 'mm_precip', 'percentage']]

401881392

In [ ]:
print(data_prepared.head().to_markdown(tablefmt = 'fancy_grid'))


╒════╤════════╤═════════╤═══════╤════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤═════════╤═════════╤═════════════════╤═══════════════╤══════════╤═════════════════╤═══════════════╤══════════╤═════════════════╤═══════════════╤══════════╤═════════════════╤═══════════════╤══════════╤═══════════════╤═════════════╤══════════════╕
│    │   year │   month │   day │   hour │   is_weekend │   timeframe1 │   timeframe2 │   timeframe3 │   timeframe4 │     lat │     lon │   temperature-4 │   mm_precip-4 │    ctx-4 │   temperature-3 │   mm_precip-3 │    ctx-3 │   temperature-2 │   mm_precip-2 │    ctx-2 │   temperature-1 │   mm_precip-1 │    ctx-1 │   temperature │   mm_precip │   percentage │
╞════╪════════╪═════════╪═══════╪════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════╪═════════╪═════════════════╪═══════════════╪══════════╪═════════════════╪═══════════════╪══════════╪═════════════════╪═══════════════╪══════════╪══

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

reduced_data_prepared = data_prepared.head(1000)

reduced_data_prepared = reduced_data_prepared[reduced_data_prepared['hour'].isin([4,9,14,19,23])]

X = reduced_data_prepared.drop(['percentage'], axis=1)
y = reduced_data_prepared['percentage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)



In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

space = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'reg_lambda': [0.1, 1.0, 10.0],
    'reg_alpha': [0, 0.1, 1.0],
    'n_estimators': [100, 200, 300]
}

xgb_model = XGBRegressor(objective='reg:squarederror', random_state = 69)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=space, scoring='neg_mean_squared_error', cv=3, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 729 candidates, totalling 2187 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=6

In [ ]:
import json

parameter = open('models/xgboost1000.json.json', 'w+')
parameter.write(json.dumps(grid_search.best_params_))
parameter.close()


In [ ]:
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=1000, max_depth=6, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, gamma=0.1, reg_alpha=0.1, reg_lambda=0.1, n_jobs=-1, random_state=123)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'RMSE: {rmse}')

# BILSTM trials


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
# Assuming your dataframe is named 'df'
# Extract the static data, the target and the time series data in seperate arrays

static_data = data_prepared[['year', 'month', 'day', 'hour', 'is_weekend', 'timeframe1','timeframe2', 'timeframe3', 'timeframe4', 'lat', 'lon', 'temperature', 'mm_precip']].values

timestep_1 = data_prepared[['temperature-1', 'mm_precip-1', 'ctx-1']].values
timestep_2 = data_prepared[['temperature-2', 'mm_precip-2', 'ctx-2']].values
timestep_3 = data_prepared[['temperature-3', 'mm_precip-3', 'ctx-3']].values
timestep_4 = data_prepared[['temperature-4', 'mm_precip-4', 'ctx-4']].values

time_series_data = np.stack((timestep_4, timestep_3, timestep_2, timestep_1), axis=1)

target = data_prepared[['percentage']].values
# timestep_1 = np.column_stack((data_prepared[['temperature-1', 'mm_precip-1', 'ctx-1']].values, static_data))
# timestep_2 = np.column_stack((data_prepared[['temperature-2', 'mm_precip-2', 'ctx-2']].values, static_data))
# timestep_3 = np.column_stack((data_prepared[['temperature-3', 'mm_precip-3', 'ctx-3']].values, static_data))
# timestep_4 = np.column_stack((data_prepared[['temperature-4', 'mm_precip-4', 'ctx-4']].values, static_data))




c:\Users\ptbad\anaconda3\lib\site-packages\dask\array\core.py:1701: FutureWarning: The `numpy.column_stack` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
c:\Users\ptbad\anaconda3\lib\site-packages\dask\array\core.py:1701: FutureWarning: The `numpy.column_stack` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
c:\Users\ptbad\anaconda3\lib\site-packages\dask\array\core.py:1701: FutureWarning: The `numpy.column_stack` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
c:\Users\ptbad\anaconda3\lib\site-packages\dask\array\core.py:1701: Futur

In [ ]:
import tensorflow as tf
from keras.layers import LSTM, Dense, Input, concatenate, Dropout

ts_input = tf.keras.Input(shape=(4, 3), name='ts_input')
static_input = tf.keras.Input(shape = (13,), name='static_input')
LSTMout = LSTM(32, activation='relu', return_sequences=False)(ts_input)
dropout_lstm = Dropout(0.2)(LSTMout)
static_out = Dense(32, activation='relu')(static_input)
dropout_static = Dropout(0.2)(static_out)

merged_out = concatenate([LSTMout, static_out])
merged_out = Dense(1, activation='relu')(merged_out)

model = tf.keras.Model(inputs=[ts_input, static_input], outputs=merged_out)

model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ts_input (InputLayer)          [(None, 4, 3)]       0           []                               
                                                                                                  
 static_input (InputLayer)      [(None, 13)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 32)           4608        ['ts_input[0][0]']               
                                                                                                  
 dense (Dense)                  (None, 32)           448         ['static_input[0][0]']           
                                                                                              

List of thing to do:
1. Encode month/hour as cyclical features or simply remove them
2. standarize percentages to max
3. standarize temperatures and rain with min max scaler
4. look into how to train with data from multiple hours and not just intervals of 4
5. F******* TRAIN SOME MODELS 
